# Long/short Trading Strategy with Support Vector Machines (SVMs)

**1. Introduction**

- The aim is to implement SVM on a trading strategy for the Luxembourg index (LUXXX) based on other indices as well as technical indicators. 
- The strategy is to take a long/short position when 0.25% change in LUXXX return is predicted. 
- The long/short position will be based on an upward or downward trending market.
- This strategy does not take into account  trading costs.

In [ ]:
import matplotlib.pyplot as plt